In [1]:
pwd

'/content'

In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Chat Bot

/content/drive/MyDrive/Colab Notebooks/Chat Bot


In [3]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Chat Bot'

In [4]:
import tensorflow as tf
from tensorflow import keras
import pickle
# libraries for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# libraries for tensorflow
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# pickle.dump({'words':words , 'classes':classes},open('wordnclass','wb'))

In [6]:
chatbot = keras.models.load_model('chatbot.pkl')

In [7]:
data = pickle.load(open('wordnclass','rb'))
words = data['words']
classes = data['classes']

In [8]:
with open('intents.json') as jsondata:
  intents = json.load(jsondata)

In [9]:
def CleanUpSentence(sentence):
  ''' This function helps create a tokenizer for every word in the given sentence and convert the word to its base form. '''
  # tokenize the pattern
  sentence_tok = nltk.word_tokenize(sentence)

  # word to stem
  sentence_tok = [stemmer.stem(w.lower()) for w in sentence_tok ]
  return sentence_tok

In [10]:
CleanUpSentence('Where are you located?')

['where', 'are', 'you', 'locat', '?']

In [11]:
def BagOfWords(sentence,words):
  ''' This function helps create a word vector for input by mapping the available words in a given sentence to pass 0*len(words). '''
  sentence_w = CleanUpSentence(sentence)

  # generating bag of words
  bag = [0] * len(words)  # words len is 52
  for word in sentence_w:
    for i,w in enumerate(words):
      if w == word: # checking that words in our sentence is available in 'words' that our model is trained on.
        # e.g first word 'where' from sentence is checked in words and if it is available then assign 1 on that place in bag array
        # similarly for 'are' 'you' 'locat' .
        bag[i] = 1

  bag = np.array(bag)
  return bag # The bag contains the accuracies of the given sentence being classified into one of the 8 classes.

In [12]:
b = BagOfWords('Where are you located',words)
b

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0])

In [13]:
res = chatbot.predict(np.array([b]))
res[0]

1/1 [==============================] - 0s 281ms/step


array([4.0252019e-02, 4.3944622e-04, 2.3221557e-03, 1.7621003e-02,
       8.0521196e-02, 8.1768185e-01, 3.8609233e-02, 2.5531619e-03],
      dtype=float32)

In [15]:
def classify(sentence):
  ''' This function helps in classifying which class the sentence belongs to. '''
  # generate probabilities from model
  bag = BagOfWords(sentence,words) # The bag contains the accuracies of the given sentence being classified into one of the 8 classes.

  results = chatbot.predict(np.array([bag])) # model will predict based on word vector the class the words belong to

  results = [[i,r] for i,r in enumerate(results[0]) if r>0.3] # this has the index and accuracy of the highest class
  # print('before sort',results)
  results.sort(key = lambda x:x[1] , reverse = True) # sorting to make sure if there are more than 1 classes that are
  # greater than 0.3 so sort according to the second element of each array, so we are sorting based on 2nd element "x[1]" in each array of array
  # that is having accuracy.
  # if we had this : [[5, 0.81768185], [2, 0.31511548], [1, 0.41768185], [8, 0.51768185]]
  # the code will sort this into : [[5.0.81768185] , [8.0.51768185] , [1.0.41768185] , [2.0.31511548]]
  # print('after sort',results)
  return_list = []

  for r in results:
    return_list.append((classes[r[0]],r[1])) # appending class and accuracy

  return return_list

In [22]:
classify('where are you located?')

1/1 [==============================] - 0s 109ms/step


[('location', 0.81768185)]

In [23]:
for a in intents['intents']:
  print(a)

{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}
{'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}
{'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}
{'tag': 'chatbot', 'patterns': ['Who built this chatbot?', 'Tell me about Chatbot', 'What is this chatbot name?'], 'responses': ['Hi, I am Chatbot designed by Bilal.', 'Thanks for asking. I am designed by Bilal Haneef.', 'I am a chatbot.']}
{'tag': 'location', 'patterns': ['What is your location?', 'Where are you located?', 'What is your address?'], 'responses': ['We are from Pakistan, the land of the pure.', 'You can visit Pakistan to meet us', 'Thanks for your Interest. I liv

In [25]:
def response(sentence):
  results = classify(sentence)
  if results: # checking if result is empty(False) or not(True)
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]: # matching the class to tag and returning one of the random response in that class.
          return random.choice(i['responses'])

      results.pop(0)

In [ ]:
response('Where are you located?')

1/1 [==============================] - 0s 24ms/step


'You can visit Pakistan to meet us'

In [ ]:
response('Hello')

1/1 [==============================] - 0s 33ms/step


'Hi there, how can I help?'

In [ ]:
response('tell me about yourself?')

1/1 [==============================] - 0s 29ms/step


'Hi, I am Bilal. Nice to meet you. I made this chatbot for fun and practice.'

In [ ]:
response('can you me some movies?')

1/1 [==============================] - 0s 38ms/step


'Inglourious Basterds'

In [ ]:
k